In [1]:
"""
Spot counts for cells with highly expressed genes (dense spots)
1. Measure total intensity of every ROI after bleed-through correction and background subtraction.
2. Calculate the number of spot from total intensity based on unitary spot fluorescence intensity
3. Correlate the number of spots (from air-localize) with the total fluorescence intensity/voxel in each ROI and determine a 'cutoff'. 
   Spot count > cutoff: use spot count converted based on total fluorescence intensity; 
   Spot count < cutoff: use spot count from Airlocalize
"""

import os, sys,z5py
import numpy as np
import pandas as pd
from glob import glob 
from skimage.measure import regionprops
from skimage.io import imread, imsave
from os.path import abspath, dirname
from scipy import stats
from scipy.stats import skewnorm,lognorm
from scipy.optimize import minimize
import itertools

In [2]:
### Specify directories to data files
input_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs"
out_dir = os.path.join(input_dir, "testfull-aug4")  
roi_csv = os.path.join(out_dir, 'roi.csv')
out_csv = os.path.join(out_dir, 'spotcount_dense_spot_corrected.csv')

# for every gene
rounds = ['r1', 'r2']
channels = ['c0', 'c1', 'c2', 'c4']

fx_spots = [os.path.join(input_dir, f'{r}\\spots\\spots_{c}.txt') 
                 for r, c in itertools.product(rounds, channels)]
fx_intns = [os.path.join(input_dir, f'{r}\\intensities\\lt171_gene_4tile_{r}_{c}_intensity.csv') 
                 for r, c in itertools.product(rounds, channels)]
fx_spcts = [os.path.join(out_dir, f'spots_{r}.csv') for r in rounds]

for f in fx_spots:
    assert os.path.isfile(f)
for f in fx_intns:
    assert os.path.isfile(f)
for f in fx_spcts:
    assert os.path.isfile(f)

In [3]:
roi = pd.read_csv(roi_csv, sep=',', index_col=0).set_index('roi')
roi

,z,y,x,area,minor_axis_length,major_axis_length,eccentricity,solidity
roi,,,,,,,,
1,363.489055,17.368291,73.106487,102.469416,8.682837,17.057071,0.509046,0.795172
2,125.553784,51.560521,396.296073,1692.922728,30.310127,42.301903,0.716519,0.836767
3,111.095382,104.351716,254.707325,1133.917848,25.492777,34.220391,0.744959,0.892238
4,55.515114,52.433029,164.453027,1471.898064,27.790082,35.438489,0.784178,0.899034
5,324.728038,91.253685,221.069806,1436.260392,28.336591,35.553770,0.797007,0.887821
...,...,...,...,...,...,...,...,...
13754,206.419432,212.102989,10.116867,234.888696,13.263912,21.561534,0.615166,0.890799
13755,252.961215,197.694189,313.463300,412.632696,15.868981,32.699733,0.485294,0.846490
13756,265.635180,65.784569,393.811699,170.012136,12.697420,20.252999,0.626940,0.702276


In [4]:
spotcount = []
for i, r in enumerate(rounds):
    f = fx_spcts[i]
    _df = pd.read_csv(f, sep=',', index_col=0)
    _df.columns = [r+"_"+col.replace('spots_', '') for col in _df.columns]
    spotcount.append(_df)
spotcount = pd.concat(spotcount, axis=1)
spotcount

,r1_c0,r1_c1,r1_c2,r1_c4,r2_c0,r2_c1,r2_c2,r2_c4
1,0.0,1.0,0.0,0.0,21.0,32.0,29.0,14.0
2,0.0,2.0,2.0,1.0,6.0,0.0,1.0,0.0
3,2.0,5.0,0.0,0.0,10.0,2.0,0.0,0.0
4,0.0,4.0,0.0,1.0,3.0,0.0,2.0,0.0
5,14.0,12.0,0.0,2.0,12.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...
13754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13755,0.0,2.0,0.0,0.0,4.0,0.0,1.0,0.0
13756,3.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0
13757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
### Identify unit spot intensity for every gene
unit_intns = []
for i, (r, c) in enumerate(itertools.product(rounds, channels)):
    f = fx_spots[i]
    print(r, c, f)
    
    # (x, y, z, I)
    spot=np.loadtxt(f, delimiter=',')
    vox=[0.92,0.92,0.84]
    spot[:,:3]=spot[:,:3]/vox  # convert from physical unit to pixel unit
    ##remove spots on edges (eliminate false detection)
    spot = spot[np.logical_and(spot[:,0]<=1500, spot[:,0]>250)]
    spot = spot[np.logical_and(spot[:,1]<=1500, spot[:,1]>250)]
    spot = spot[np.logical_and(spot[:,2]<=650,  spot[:,2]>150)]   
    
    ## assign the most frequent intensity as the single-spot-intensity
    spot_int = spot[:,3]
    spot_int = spot_int[spot_int!=-8.0] # ???
    n,b=np.histogram(spot_int, bins=5000)
    unit_intn = b[np.argmax(n)]
    unit_intns.append(unit_intn)
    
unit_intns

r1 c0 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\spots\spots_c0.txt
r1 c1 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\spots\spots_c1.txt
r1 c2 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\spots\spots_c2.txt
r1 c4 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\spots\spots_c4.txt
r2 c0 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c0.txt
r2 c1 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c1.txt
r2 c2 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c2.txt
r2 c4 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c4.txt


[134.68825055999997,
 142.20793256000002,
 137.06713656,
 137.09174240000002,
 134.17228896,
 135.83476556,
 132.0431733,
 131.39002394]

In [6]:
# df_count is spot count calculated from total fluorescence intensity
df_count = pd.DataFrame(index=roi.index, dtype=float)
for i, (r, c) in enumerate(itertools.product(rounds, channels)):
    f = fx_intns[i]
    unit_int = unit_intns[i]
    print(r, c, f)

    cell_int = pd.read_csv(f, sep=',', index_col=0)
    cell_int = cell_int.reindex(roi.index) ## only include intact ROIs###

    vec_mean = cell_int['mean_intensity'].values
    vec_area = roi['area'].values

    # background
    n,b = np.histogram(vec_mean, bins=1000)
    bg = b[np.argmax(n)]    
    
    # count
    vec_count = np.clip(vec_mean - bg, 0, None)*vec_area/unit_int
    df_count[f'{r}_{c}'] = vec_count

df_count

r1 c0 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\intensities\lt171_gene_4tile_r1_c0_intensity.csv
r1 c1 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\intensities\lt171_gene_4tile_r1_c1_intensity.csv
r1 c2 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\intensities\lt171_gene_4tile_r1_c2_intensity.csv
r1 c4 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\intensities\lt171_gene_4tile_r1_c4_intensity.csv
r2 c0 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\intensities\lt171_gene_4tile_r2_c0_intensity.csv
r2 c1 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\intensities\lt171_gene_4tile_r2_c1_intensity.csv
r2 c2 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\intensities\lt171_gene_4tile_r2_c2_intensity.csv
r2 c4 D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\intensities\lt171_gene_4tile_r2_c4_intensity.csv


,r1_c0,r1_c1,r1_c2,r1_c4,r2_c0,r2_c1,r2_c2,r2_c4
roi,,,,,,,,
1,159.088138,1.696923,128.996451,27.756485,111.398637,27.950531,136.629569,120.947143
2,0.633554,0.000000,0.000000,0.000000,1.910364,18.274678,3.632611,1.005871
3,2.779255,6.333779,1.746320,1.917305,6.130914,18.657606,2.961792,3.106435
4,1.108907,0.000000,0.000000,0.000000,0.175846,19.608370,2.802156,0.000000
5,1.623296,4.023971,1.140955,2.229959,7.082793,21.452617,6.634511,4.027236
...,...,...,...,...,...,...,...,...
13754,0.000000,0.000000,0.000000,0.000000,0.000000,1.987056,0.075685,0.000000
13755,0.000000,6.526348,0.000000,0.000000,0.000000,6.650384,0.000000,0.000000
13756,0.000000,0.000000,0.000000,0.000000,0.000000,2.414219,0.234022,0.000000


In [7]:
# proc and save
# update spotcount using df_count
df_cutoff = spotcount.copy()
# loop over genes
for i in df_count.columns:
    density = spotcount[i]/(roi['area']*2*2*2/(0.92*0.92*0.84)) # convert um^3 to voxel values
    cond = density > 0.01 ##this threshold corresponds to spot-spot distance ~1.3 um apart
    cond
    print(i, cond.sum())
    df_cutoff.loc[cond, i] = df_count.loc[cond, i].copy()
    
df_cutoff

r1_c0 1
r1_c1 9
r1_c2 1
r1_c4 0
r2_c0 7
r2_c1 51
r2_c2 7
r2_c4 7


,r1_c0,r1_c1,r1_c2,r1_c4,r2_c0,r2_c1,r2_c2,r2_c4
1,0.0,1.0,0.0,0.0,111.398637,27.950531,136.629569,120.947143
2,0.0,2.0,2.0,1.0,6.000000,0.000000,1.000000,0.000000
3,2.0,5.0,0.0,0.0,10.000000,2.000000,0.000000,0.000000
4,0.0,4.0,0.0,1.0,3.000000,0.000000,2.000000,0.000000
5,14.0,12.0,0.0,2.0,12.000000,2.000000,2.000000,1.000000
...,...,...,...,...,...,...,...,...
13754,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
13755,0.0,2.0,0.0,0.0,4.000000,0.000000,1.000000,0.000000
13756,3.0,4.0,0.0,0.0,1.000000,0.000000,0.000000,0.000000
13757,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [8]:
df_cutoff.to_csv(out_csv)